# Subreddit level data collection

Each code chunk will execute one step in the process of collecting subreddit level data, including information on its moderators and the rules inputed into its Automoderator. Part 1 will first get us a full list of subreddit names starting from the most recent subreddit created all the way back to /r/reddit.com, which should always be at the end of the list. In Part 2, we will then iterate over this list to crawl each subreddit's /r/(subreddit)/about.json, r/(subreddit)/about/moderators.json, and r/(subreddit)/about/rules.json. 

### 1. Get list of all <span style="color:red"> subreddit names</span>

Note: change the custom user agent and script variables to your liking.        

Also, change the `times_run` variable to a number like `30,000`. This should sufficiently get the scraper to go back 30,000 pages in time in subreddit names. It will not actually do 30,000 web crawls---the code will stop probably around 20,000, as the number of subreddits is close to 2,000,000 (each page will contain 100 subreddits, so 30,000 * 100 = 3 million as a safe upperbound).

In [1]:
import requests
import csv
import time
from datetime import datetime
from datetime import timedelta
from requests_toolbelt import user_agent
from requests import Session
import re
from pprint import pprint
import pandas as pd

# Prepare output file with date of scrape
today = datetime.today()
output_file = "subreddit_list-" + today.strftime("%d-%m-%Y") + ".tsv"
print(output_file)

# Enter your custom script and user agent information
user_agent_name = "healspersecond_ckiene_uw-com"
script_name = "subreddit_list_creator"
script_vers = "1"
my_script = "{}/{}".format(script_name, script_vers)

s = Session()
s.headers = { 
    'User-Agent': user_agent(user_agent_name, my_script)
    }
print(s.headers)

# Define functions to parse json 
header = ['display_name']

def parse_response(User):
    pprint(display_name())
    return data

def display_name():
    result = data['data']['children'][n]['data']['display_name']
    return result

results = [] 
dict = {}

# The URL to directly crawl the API---feel free to change it to your desired API endpoint
request_url = 'https://www.reddit.com/subreddits/new.json?limit=100'
r = requests.get(request_url, headers= s.headers)
data = r.json()
after_token = data['data']['after']

sub_number = len(data['data']['children'])
sub_num_seq = list(range(0,sub_number))

for item in header:
    for n in sub_num_seq :
        dict[item] = data['data']['children'][n]['data'][item]
        results.append(dict)#[item])

# Specify number of pages to crawl back
times_run =  3

# Note: 30,000 will crawl back far enough to capture all subreddits and is suggested

with open(output_file, 'w') as csvfile:
    fieldnames = results[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    i = 1
    while i <= times_run:
        if after_token == None:
            print("End of list at: ",i-1)
            break
        else:
            request_url = 'https://www.reddit.com/subreddits/new.json?limit=100&after='+after_token
            response = requests.get(request_url)
            if response.status_code != 200: 
                continue
            else:
                r = requests.get(request_url, headers= s.headers)
                data = r.json()
                sub_number = len(data['data']['children'])
                sub_num_seq = list(range(0,sub_number))
                after_token = data['data']['after']
                   
                for item in header:
                    for n in sub_num_seq:
                        dict[item] = data['data']['children'][n]['data'][item]
                        results.append(dict[item])
                        writer.writerow(dict)  #add to list
                print("Request ", i, " completed! after_token: ", after_token) 
                i = i + 1

# If things break, the last after_token can be used in the request_url to pick up where you left off

subreddit_list-02-11-2020.tsv
{'User-Agent': 'healspersecond_ckiene_uw-com/subreddit_list_creator/1 CPython/3.7.4 Windows/10'}
Request  1  completed! after_token:  t5_3cgm97
Request  2  completed! after_token:  t5_3cg7uh
Request  3  completed! after_token:  t5_3cfu1r


In [3]:
pd.read_table(output_file)

,display_name
0,DidEDigUniPD20
1,BuildsofSkyrim
2,lexmeme
3,suchblank
4,Desolate_Era
...,...
295,GenshinTheoryCraft
296,MemeinVietnam
297,DarkMayhem65t
298,HOABattles


### 2. Crawl reddit URLs and store <span style="color:red">JSON</span> for each subreddit to <span style="color:red">SQLite3 db</span> 

In [6]:
import requests
import sqlite3
import csv
import time
import pandas as pd
import re
from requests import Session
from requests_toolbelt import user_agent
import datetime
from datetime import timedelta
import json

conn = sqlite3.connect("test_db.db")

# List of subreddits to search
sublist = pd.read_table(output_file, header = 0)
sublist = sublist.display_name.tolist()

# Time reporting variables
i = 1
start = time.time()
list_len = len(sublist)

       
for sub in sublist:              
    print(i, '/', list_len , sub)
    
    # JSON requests - subreddit info
    dict_sub = {}
    request_url_sub = 'https://www.reddit.com/r/'+sub+'/about.json'     
    r_sub = requests.get(request_url_sub, headers = s.headers)
    
    try:
        data_sub = r_sub.json()
        sub_json_status = "OK"
    except json.decoder.JSONDecodeError as err:
        data_sub = str(err)
        sub_json_status = str(err)
    
    # JSON requests - moderator info
    dict_mod = {}
    request_url_mod = 'https://www.reddit.com/r/'+sub+'/about/moderators.json'     
    r_mod = requests.get(request_url_mod, headers = s.headers)

    try:
        data_mod = r_mod.json()
        mod_json_status = "OK"
    except json.decoder.JSONDecodeError as err:
        data_mod = str(err)
        mod_json_status = str(err)    

    # JSON requests - rules info
    dict_rules = {}
    request_url_rules = 'https://www.reddit.com/r/'+sub+'/about/rules.json'     
    r_rules = requests.get(request_url_rules, headers = s.headers)

    try:
        data_rules = r_rules.json()
        rules_json_status = "OK"
    except json.decoder.JSONDecodeError as err:
        data_rules = str(err)
        rules_json_status = str(err)
        
    timestamp = datetime.datetime.utcfromtimestamp(time.time()).strftime("%Y-%m-%d %H:%M:%S")
                
    # Write to dictionary for subreddit table
    dict_sub['subreddit'] = [sub]
    dict_sub['sub_status'] = [r_sub.status_code]
    dict_sub['headers'] = [str(r_sub.headers)]
    dict_sub['sub_reason'] = [r_sub.reason]
    dict_sub['sub_timestamp'] = [timestamp]
    dict_sub['sub_json_status'] = [sub_json_status]
    dict_sub['sub_json'] = [json.dumps(data_sub)]
    
    # Write to dictionary for mod table
    dict_mod['subreddit'] = [sub]
    dict_mod['mod_status'] = [r_mod.status_code]
    dict_mod['headers'] = [str(r_mod.headers)]
    dict_mod['mod_reason'] = [r_mod.reason]
    dict_mod['mod_timestamp'] = [timestamp]
    dict_mod['mod_json_status'] = [mod_json_status]
    dict_mod['mod_json'] = [json.dumps(data_mod)]
    
    # Write to dictionary for rules table
    dict_rules['subreddit'] = [sub]
    dict_rules['rules_status'] = [r_rules.status_code]
    dict_rules['headers'] = [str(r_rules.headers)]
    dict_rules['rules_reason'] = [r_rules.reason]
    dict_rules['rules_timestamp'] = [timestamp]
    dict_rules['rules_json_status'] = [rules_json_status]
    dict_rules['rules_json'] = [json.dumps(data_rules)]
               
    pd_sub = pd.DataFrame(dict_sub)
    pd_sub.to_sql('sub_json', if_exists='append',con = conn, index=False)
    
    pd_mod = pd.DataFrame(dict_mod)
    pd_mod.to_sql('mod_json', if_exists='append',con = conn, index=False)
    
    pd_rules = pd.DataFrame(dict_rules)
    pd_rules.to_sql('rules_json', if_exists='append',con = conn, index=False)
    
    # Time reporting
    finish = time.time()
    list_len = len(sublist)
    list_len_remain = list_len - i
    seconds_per_sub = (finish - start) / i
    seconds_remaining = list_len_remain * seconds_per_sub
    print(list_len_remain, 'subreddits remaining.') 
    sec = timedelta(seconds=(seconds_remaining))
    print(str(sec), "time remaining.")
    time.sleep(0.5)
    
    i =  i + 1   

1 / 300 DidEDigUniPD20
299 subreddits remaining.
0:08:53.575303 time remaining.
2 / 300 BuildsofSkyrim
298 subreddits remaining.
0:07:25.296393 time remaining.
3 / 300 lexmeme
297 subreddits remaining.
0:07:04.728265 time remaining.
4 / 300 suchblank
296 subreddits remaining.
0:06:48.460785 time remaining.
5 / 300 Desolate_Era
295 subreddits remaining.
0:06:48.040449 time remaining.
6 / 300 SumnerMemorialHS
294 subreddits remaining.
0:07:06.102451 time remaining.
7 / 300 Throwbackk
293 subreddits remaining.
0:06:53.527020 time remaining.
8 / 300 surprise_crochet
292 subreddits remaining.
0:06:45.023361 time remaining.
9 / 300 Blacklyte
291 subreddits remaining.
0:06:37.718253 time remaining.
10 / 300 cryptonix
290 subreddits remaining.
0:06:40.950356 time remaining.
11 / 300 UsefulBookshop101
289 subreddits remaining.
0:06:34.968714 time remaining.
12 / 300 NickiOsak
288 subreddits remaining.
0:06:54.223755 time remaining.
13 / 300 WokeJusticeWarriorz
287 subreddits remaining.
0:06:53.

196 subreddits remaining.
0:05:11.443872 time remaining.
105 / 300 weebsANDgamers
195 subreddits remaining.
0:05:10.214288 time remaining.
106 / 300 eatshitapple
194 subreddits remaining.
0:05:07.912766 time remaining.
107 / 300 AskMonkeys
193 subreddits remaining.
0:05:05.639006 time remaining.
108 / 300 OhSoYouLike
192 subreddits remaining.
0:05:03.346754 time remaining.
109 / 300 TeenageRants
191 subreddits remaining.
0:05:01.377350 time remaining.
110 / 300 BloodyEvil
190 subreddits remaining.
0:05:02.935361 time remaining.
111 / 300 MCBloodAndSteel
189 subreddits remaining.
0:05:02.584959 time remaining.
112 / 300 RogerWakefieldPosts
188 subreddits remaining.
0:05:00.377321 time remaining.
113 / 300 randomnessandlaughs
187 subreddits remaining.
0:04:58.108972 time remaining.
114 / 300 advertise_anything
186 subreddits remaining.
0:04:55.853781 time remaining.
115 / 300 AM2Kofficial
185 subreddits remaining.
0:04:53.722593 time remaining.
116 / 300 FuckCrossplay
184 subreddits rema

205 / 300 DoomViles
95 subreddits remaining.
0:02:28.029161 time remaining.
206 / 300 processmanagement
94 subreddits remaining.
0:02:26.530165 time remaining.
207 / 300 gsnee62
93 subreddits remaining.
0:02:25.336012 time remaining.
208 / 300 AdviceaboutEHR
92 subreddits remaining.
0:02:23.606405 time remaining.
209 / 300 TAWOTA
91 subreddits remaining.
0:02:21.873943 time remaining.
210 / 300 TF2_hat_ideas
90 subreddits remaining.
0:02:20.148154 time remaining.
211 / 300 WeldingMasters
89 subreddits remaining.
0:02:18.742988 time remaining.
212 / 300 DronesUAV
88 subreddits remaining.
0:02:17.077693 time remaining.
213 / 300 churchofgeg
87 subreddits remaining.
0:02:15.372062 time remaining.
214 / 300 RickRollGang
86 subreddits remaining.
0:02:13.697744 time remaining.
215 / 300 AndyGillion
85 subreddits remaining.
0:02:11.994423 time remaining.
216 / 300 PeeCumMemez
84 subreddits remaining.
0:02:10.356875 time remaining.
217 / 300 fromrussia
83 subreddits remaining.
0:02:08.782408 t

Examine the tables we've created:

In [7]:
# Subreddit's general information
df_sub = pd.read_sql_query("select * from sub_json;", conn)
df_sub.head()

,subreddit,sub_status,headers,sub_reason,sub_timestamp,sub_json_status,sub_json
0,DidEDigUniPD20,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:54,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
1,BuildsofSkyrim,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:55,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
2,lexmeme,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:56,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
3,suchblank,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:57,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
4,Desolate_Era,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:59,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."


In [8]:
# Subreddit's moderator information
df_mod = pd.read_sql_query("select * from mod_json;", conn)
df_mod.head()

,subreddit,mod_status,headers,mod_reason,mod_timestamp,mod_json_status,mod_json
0,DidEDigUniPD20,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:54,OK,"{""kind"": ""UserList"", ""data"": {""children"": [{""n..."
1,BuildsofSkyrim,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:55,OK,"{""kind"": ""UserList"", ""data"": {""children"": [{""n..."
2,lexmeme,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:56,OK,"{""kind"": ""UserList"", ""data"": {""children"": [{""n..."
3,suchblank,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:57,OK,"{""kind"": ""UserList"", ""data"": {""children"": [{""n..."
4,Desolate_Era,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:59,OK,"{""kind"": ""UserList"", ""data"": {""children"": [{""n..."


In [9]:
# Subreddit's rules information
df_rules  = pd.read_sql_query("select * from rules_json;", conn)
df_rules.head()

,subreddit,rules_status,headers,rules_reason,rules_timestamp,rules_json_status,rules_json
0,DidEDigUniPD20,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:54,OK,"{""rules"": [], ""site_rules"": [""Spam"", ""Personal..."
1,BuildsofSkyrim,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:55,OK,"{""rules"": [], ""site_rules"": [""Spam"", ""Personal..."
2,lexmeme,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:56,OK,"{""rules"": [], ""site_rules"": [""Spam"", ""Personal..."
3,suchblank,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:57,OK,"{""rules"": [], ""site_rules"": [""Spam"", ""Personal..."
4,Desolate_Era,200,"{'Connection': 'keep-alive', 'Content-Length':...",OK,2020-11-02 18:06:59,OK,"{""rules"": [], ""site_rules"": [""Spam"", ""Personal..."


With this code, we can generate a full and current list of subreddits and then used that list to collect the JSON on each of those subreddits. In the next section, I'll show you what this json data looks like when structured as a new dataframe using a new example of city related subreddits.

### 3. JSON <span style="color:red">data transformation</span> for each table's JSON column

First, I'm going to add rows to our tables in our SQL db for some specific subreddits that I know will have populated JSON with moderator and rule information. Newer subreddits seem less likely to have this, so the ones we scraped in the code above would not be a great example to show what the final data set looks like.

In [10]:
import requests
import sqlite3
import csv
import time
import pandas as pd
import re
from requests import Session
from requests_toolbelt import user_agent
import datetime
from datetime import timedelta
import json

conn = sqlite3.connect("city_test.db")

city_subs = ('seattlewa', 'nyc', 'chicago', 'losangeles', 'atlanta')

for sub in city_subs:                
    # JSON requests - subreddit info
    print(sub)
    dict_sub = {}
    request_url_sub = 'https://www.reddit.com/r/'+sub+'/about.json'     
    r_sub = requests.get(request_url_sub, headers = s.headers)
    
    try:
        data_sub = r_sub.json()
        sub_json_status = "OK"
    except json.decoder.JSONDecodeError as err:
        data_sub = str(err)
        sub_json_status = str(err)
    
    # JSON requests - moderator info
    dict_mod = {}
    request_url_mod = 'https://www.reddit.com/r/'+sub+'/about/moderators.json'     
    r_mod = requests.get(request_url_mod, headers = s.headers)

    try:
        data_mod = r_mod.json()
        mod_json_status = "OK"
    except json.decoder.JSONDecodeError as err:
        data_mod = str(err)
        mod_json_status = str(err)    

    # JSON requests - rules info
    dict_rules = {}
    request_url_rules = 'https://www.reddit.com/r/'+sub+'/about/rules.json'     
    r_rules = requests.get(request_url_rules, headers = s.headers)

    try:
        data_rules = r_rules.json()
        rules_json_status = "OK"
    except json.decoder.JSONDecodeError as err:
        data_rules = str(err)
        rules_json_status = str(err)
        
    timestamp = datetime.datetime.utcfromtimestamp(time.time()).strftime("%Y-%m-%d %H:%M:%S")
                
    # Write to dictionary for subreddit table
    dict_sub['subreddit'] = [sub]
    dict_sub['sub_status'] = [r_sub.status_code]
    dict_sub['sub_reason'] = [r_sub.reason]
    dict_sub['sub_timestamp'] = [timestamp]
    dict_sub['sub_json_status'] = [sub_json_status]
    dict_sub['sub_json'] = [json.dumps(data_sub)]
    
    # Write to dictionary for mod table
    dict_mod['subreddit'] = [sub]
    dict_mod['mod_status'] = [r_mod.status_code]
    dict_mod['mod_reason'] = [r_mod.reason]
    dict_mod['mod_timestamp'] = [timestamp]
    dict_mod['mod_json_status'] = [mod_json_status]
    dict_mod['mod_json'] = [json.dumps(data_mod)]
    
    # Write to dictionary for rules table
    dict_rules['subreddit'] = [sub]
    dict_rules['rules_status'] = [r_rules.status_code]
    dict_rules['rules_reason'] = [r_rules.reason]
    dict_rules['rules_timestamp'] = [timestamp]
    dict_rules['rules_json_status'] = [rules_json_status]
    dict_rules['rules_json'] = [json.dumps(data_rules)]
               
    pd_sub = pd.DataFrame(dict_sub)
    pd_sub.to_sql('sub_json', if_exists='append',con = conn, index=False)
    
    pd_mod = pd.DataFrame(dict_mod)
    pd_mod.to_sql('mod_json', if_exists='append',con = conn, index=False)
    
    pd_rules = pd.DataFrame(dict_rules)
    pd_rules.to_sql('rules_json', if_exists='append',con = conn, index=False)

seattlewa
nyc
chicago
losangeles
atlanta


### General subreddit information

For the `sub_json table`, we can use the `json_normalize` pandas command to transform the `sub_json` column into a new, fully populated data set.

In [11]:
# Subreddit's general information
df_sub = pd.read_sql_query("select * from sub_json;", conn)
df_sub

,subreddit,sub_status,sub_reason,sub_timestamp,sub_json_status,sub_json
0,seattlewa,200,OK,2020-11-02 18:15:14,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
1,nyc,200,OK,2020-11-02 18:15:15,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
2,chicago,200,OK,2020-11-02 18:15:16,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
3,losangeles,200,OK,2020-11-02 18:15:17,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."
4,atlanta,200,OK,2020-11-02 18:15:17,OK,"{""kind"": ""t5"", ""data"": {""user_flair_background..."


In [12]:
from pandas.io.json import json_normalize

# First, load the json:
df_sub['sub_json'] = df_sub['sub_json'].apply(json.loads)

# Add a subreddit name key to the JSON dictionary
for i in df_sub.index.tolist():
        df_sub.sub_json[i]['data']['subreddit'] = df_sub.subreddit[i]

final_subdf = json_normalize(df_sub.sub_json)
final_subdf.columns = final_subdf.columns.str.lstrip('data.') # remove the 'data.' prefix

# Reorder columns
cols = final_subdf.columns.tolist()
cols = cols[-1:] + cols[:-1]
final_subdf = final_subdf[cols]
final_subdf = final_subdf.reset_index(drop=True)
# Transform created_utc column to datetime
final_subdf['created_utc'] = final_subdf['created_utc'].apply(datetime.datetime.fromtimestamp)
final_subdf

,subreddit,kind,user_flair_background_color,submit_text_html,restrict_posting,user_is_banned,free_form_reports,wiki_enabled,user_is_muted,user_can_flair_in_sr,...,restrict_commenting,user_flair_css_class,llow_images,lang,whitelist_status,url,created_utc,banner_size,mobile_banner_image,user_is_contributor
0,seattlewa,t5,None,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",True,None,True,True,None,None,...,False,None,True,en,all_ads,/r/SeattleWA/,2012-10-17 12:37:09,"[653, 196]",,None
1,nyc,t5,None,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",True,None,True,True,None,None,...,False,None,True,en,all_ads,/r/nyc/,2008-04-17 16:45:27,None,,None
2,chicago,t5,None,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",True,None,False,True,None,None,...,False,None,True,en,all_ads,/r/chicago/,2008-01-24 22:22:33,"[1280, 384]",,None
3,losangeles,t5,None,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",True,None,True,True,None,None,...,False,None,True,en,all_ads,/r/LosAngeles/,2008-04-14 03:06:50,None,https://styles.redditmedia.com/t5_2qht0/styles...,None
4,atlanta,t5,None,None,True,None,True,True,None,None,...,False,None,True,en,all_ads,/r/Atlanta/,2008-06-11 11:53:55,"[1280, 384]",,None


#### Note: 94 columns is not ideal---there are many columns that have mostly useless information or are just copies of other columns but in html format. When I normalize the subreddit/about.json full data set, I only pull out the `description_html`, `subscribers`, `created`, and `display_name` of each subreddit.

### Moderator information

The actual moderator information here is nested deeper within the JSON object, so we will need to do some preliminary work before we can use the `json_normalize` command here.

In [13]:
# Subreddit's general information
df_mod = pd.read_sql_query("select * from mod_json;", conn)
df_mod['mod_json'] = df_mod['mod_json'].apply(json.loads) # load json
df_mod

,subreddit,mod_status,mod_reason,mod_timestamp,mod_json_status,mod_json
0,seattlewa,200,OK,2020-11-02 18:15:14,OK,"{'kind': 'UserList', 'data': {'children': [{'n..."
1,nyc,200,OK,2020-11-02 18:15:15,OK,"{'kind': 'UserList', 'data': {'children': [{'n..."
2,chicago,200,OK,2020-11-02 18:15:16,OK,"{'kind': 'UserList', 'data': {'children': [{'n..."
3,losangeles,200,OK,2020-11-02 18:15:17,OK,"{'kind': 'UserList', 'data': {'children': [{'n..."
4,atlanta,200,OK,2020-11-02 18:15:17,OK,"{'kind': 'UserList', 'data': {'children': [{'n..."


In [14]:
# Add the subreddit name to the dictionary so we can use it as a key with the other data sets
for i in df_mod.index.tolist():
    children_len = len(df_mod.mod_json[i]['data']['children'])
    children_range = list(range(0, children_len))
    for n in children_range:
        df_mod.mod_json[i]['data']['children'][n]['subreddit'] = df_mod.subreddit[i]

mod_data = df_mod.mod_json.to_list()
df_mod = pd.DataFrame(columns=list(mod_data[0]['data']['children'][0].keys()))
mod_children_range = list(range(0, len(mod_data)))

# Normalize the JSON and append to df_mod
for i in mod_children_range:
    df_mod = df_mod.append(json_normalize(mod_data[i]['data']['children']))
    
# Reorder columns and fix column names                      
cols = df_mod.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_mod = df_mod[cols]
df_mod = df_mod.reset_index(drop=True)
# Transform created_utc column to datetime
df_mod['date'] = df_mod['date'].apply(datetime.datetime.fromtimestamp)
df_mod

,subreddit,name,author_flair_text,mod_permissions,date,rel_id,id,author_flair_css_class
0,seattlewa,rattus,:snoo_tableflip: :table_flip:,[all],2015-01-15 12:43:18,rb_8sfnq6,t2_vahn,plain
1,seattlewa,isiramteal,anti-Taco timers OUT 😡👉🚪,[all],2015-03-04 23:42:26,rb_9h4tc9,t2_lpoh8,plain
2,seattlewa,ExtraNoise,None,"[wiki, posts, mail, flair]",2016-06-08 21:20:04,rb_g9vt44,t2_4bg92,None
3,seattlewa,YopparaiNeko,Greenlake,[all],2016-07-16 13:34:33,rb_h3cqe0,t2_gheks,plain
4,seattlewa,AutoModerator,None,[all],2016-09-06 15:44:06,rb_ibk175,t2_6l4z3,None
...,...,...,...,...,...,...,...,...
57,atlanta,cosby,McDonough,[all],2013-01-31 19:40:08,rb_2k0c87,t2_3hftj,None
58,atlanta,AutoModerator,ITP,"[wiki, posts, flair]",2013-05-29 05:59:17,rb_38z9su,t2_6l4z3,None
59,atlanta,oakgrove,,[all],2016-03-07 16:01:45,rb_ekfobc,t2_67qn4,
60,atlanta,askatlmod,,"[wiki, posts, mail, flair]",2018-10-18 15:56:12,rb_11khvgc,t2_1npuwp6p,None


### Rules information

We'll do similar work that we did on the moderator table with the rules table

In [15]:
# Subreddit's general information
df_rules = pd.read_sql_query("select * from rules_json;", conn)
df_rules['rules_json'] = df_rules['rules_json'].apply(json.loads) # load json
df_rules

,subreddit,rules_status,rules_reason,rules_timestamp,rules_json_status,rules_json
0,seattlewa,200,OK,2020-11-02 18:15:14,OK,"{'rules': [{'kind': 'link', 'description': 'Su..."
1,nyc,200,OK,2020-11-02 18:15:15,OK,"{'rules': [{'kind': 'all', 'description': '', ..."
2,chicago,200,OK,2020-11-02 18:15:16,OK,"{'rules': [{'kind': 'link', 'description': 'Pl..."
3,losangeles,200,OK,2020-11-02 18:15:17,OK,"{'rules': [{'kind': 'link', 'description': 'Re..."
4,atlanta,200,OK,2020-11-02 18:15:17,OK,"{'rules': [{'kind': 'all', 'description': 'No ..."


In [16]:
# First, add 'subreddit' key to the JSON dict
for i in df_rules.index.tolist():
    df_rules.rules_json[i]['subreddit'] = df_rules.subreddit[i]

rules_data = df_rules.rules_json.tolist()

# Normalize the JSON and append to df_rules
df_rules = json_normalize(rules_data, record_path=['rules'], meta=['subreddit'])

# Reorder columns and fix column names   
cols = df_rules.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_rules = df_rules[cols]
df_rules.columns = df_rules.columns.str.replace("data.","")
df_rules = df_rules.reset_index(drop=True)
# Transform created_utc column to datetime
df_rules['created_utc'] = df_rules['created_utc'].apply(datetime.datetime.fromtimestamp)
df_rules

,subreddit,kind,description,short_name,violation_reason,created_utc,priority,description_html
0,seattlewa,link,Submissions should be on topic to Seattle and ...,Only Seattle and Puget Sound Related Submissions,No explicit impact or connection to Seattle.,2017-03-07 17:33:46,0,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
1,seattlewa,all,This discussion board promotes civil discourse...,No Personal Attacks,No Personal Attacks,2017-03-07 17:34:17,1,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
2,seattlewa,all,If a user is seen to regularly act in bad fai...,Challenges,Challenges,2018-05-02 12:14:19,2,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
3,seattlewa,all,Moderators must enforce Reddit’s site-wide rul...,Follow Reddit’s Content Policy,Breaking Reddit's Content Policy,2018-05-02 12:15:03,3,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
4,seattlewa,link,"Reddit is supposed to catch duplicate posts, b...",Duplicate Post Source,100% Duplicate Post Source,2019-02-19 09:46:25,4,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
5,nyc,all,,"Be nice, or don't comment. Rudeness, meanness,...","Not nice, rude, mean, or otherwise unwelcoming...",2016-01-28 02:50:17,0,NaN
6,nyc,link,"**Photos** of WTC1, sunsets, and other common ...",Questions go to /r/AskNYC. Pictures go into /r...,Miscategorized.,2016-01-28 02:51:37,1,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
7,nyc,all,"Please do not post “shortened” links, redirect...",No link shorteners or redirects.,No link shorteners or redirects.,2016-01-28 02:52:58,2,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
8,nyc,link,"Spamblogs, pages heavy with ads, and anything ...",No spamblogs and no ad-heavy pages,No spamblogs and no ad-heavy pages,2016-01-28 02:55:17,3,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt..."
9,nyc,comment,,Don't shit up comment threads,Unnecessary shit in comments,2016-01-28 08:06:43,4,NaN
